In [31]:
import os
import numpy as np
import polars as pl
from lets_plot import *
LetsPlot.setup_html()

In [50]:
def decay_plot(df:pl.DataFrame):

    p = (
        ggplot(df,aes(x='timestep',y='counts',color=as_discrete('um')))
        + geom_point(size=4)
        + geom_line(size=1.2)
        + scale_color_hue()
    )
    return p

In [51]:
df = pl.read_csv("merged.csv")

In [52]:
df.head()

10_left,10_right,10_all,20_left,20_right,20_all,40_left,40_right,40_all,60_left,60_right,60_all,timestep
i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64
95,103,105,190,193,205,409,414,419,613,617,624,1
25,27,44,56,58,85,202,192,287,268,283,404,2
9,7,20,15,17,38,112,114,208,168,173,291,3
1,1,6,5,3,16,67,69,141,114,113,228,4
1,1,2,5,2,12,42,48,110,84,86,194,5


In [59]:
for col in df.columns:
    if col != "timestep":
        df = df.with_columns(
            ((pl.col(col)/pl.col(col).max())*100)
        ) 

In [66]:
df_melted = df.head(20).melt(id_vars="timestep",value_name='counts',variable_name="case")
df_melted.head()

C:\Users\zafi_\AppData\Local\Temp\ipykernel_12504\932811825.py:1: DeprecationWarning: `DataFrame.melt` is deprecated. Use `unpivot` instead, with `index` instead of `id_vars` and `on` instead of `value_vars`
  df_melted = df.head(20).melt(id_vars="timestep",value_name='counts',variable_name="case")


timestep,case,counts
i64,str,f64
1,"""10_left""",100.0
2,"""10_left""",26.315789
3,"""10_left""",9.473684
4,"""10_left""",1.052632
5,"""10_left""",1.052632


In [67]:
dfm = df_melted.with_columns(
    pl.col("case").str.split("_").list.get(0).cast(pl.UInt16).alias("um"),
    pl.col("case").str.split("_").list.get(1).alias("which"),
)

In [68]:
dfm

timestep,case,counts,um,which
i64,str,f64,u16,str
1,"""10_left""",100.0,10,"""left"""
2,"""10_left""",26.315789,10,"""left"""
3,"""10_left""",9.473684,10,"""left"""
4,"""10_left""",1.052632,10,"""left"""
5,"""10_left""",1.052632,10,"""left"""
…,…,…,…,…
16,"""60_all""",2.24359,60,"""all"""
17,"""60_all""",1.762821,60,"""all"""
18,"""60_all""",1.762821,60,"""all"""


In [69]:
dfm_whole = dfm.filter(pl.col("which") == "all")

In [70]:
dfm_whole

timestep,case,counts,um,which
i64,str,f64,u16,str
1,"""10_all""",100.0,10,"""all"""
2,"""10_all""",41.904762,10,"""all"""
3,"""10_all""",19.047619,10,"""all"""
4,"""10_all""",5.714286,10,"""all"""
5,"""10_all""",1.904762,10,"""all"""
…,…,…,…,…
16,"""60_all""",2.24359,60,"""all"""
17,"""60_all""",1.762821,60,"""all"""
18,"""60_all""",1.762821,60,"""all"""


In [71]:
decay_plot(dfm_whole)